# Option pricing

The goal here is to do an option pricing exercise. Inspired by [this blog post](http://firsttimeprogrammer.blogspot.co.uk/2015/08/european-option-pricing-with-python.html).

## Definitions and concepts

### Statistics

Let $p(X)$ be the probability of a random variable. Replace sums with integrals involving the probability distribution function in the continuous case.

Mean (expected value) is $E[X] = \sum_{k=1}^n p(x_k)x_k$.

Variance is $\mathrm{Var}[X] = E\left[ (X - E[X])^2 \right] = E[X^2] - E[X]^2$.

### Geometric Brownian motion

### Stock price drift and variance

Define a *return* $r_i$ at time $t_i$ given prices $S_i$ and $S_{i-1}$ where $t_i - t_{i-1} > 0$ as
$$ r_i = \frac{p_i - S_{i-1}}{S_{i-1}}. $$
This measures *percent* change. Then $\log(1 + r_i)$ is normally distributed assuming prices are in a log normal distribution since
$$ 1 + r_i = \frac{S_i}{S_{i-1}} = \exp^{\log\left(\frac{S_i}{S_{i-1}}\right)}. $$

Drift ($\mu$) and volatility ($\sigma$) are defined in terms of expected value ($U$) and variance ($V^2$) of the log of price ratios. Let $n$ be the number of trading days (number of price points), which means $\Delta t = 1/n$. Then
$$ U = \frac{1}{n-1} \sum_{i=1}^{n-1} \log\left( \frac{S_{i+1}}{S_i} \right) $$
and
$$ V^2 = \frac{1}{n-2} \sum_{i=1}^{n-1} \left( \log\left( \frac{S_{i+1}}{S_i} \right) - U \right)^2. $$
Note the last is a *sample* variance. Volatility and drift are then
$$ \mu = \frac{U + V^2/2}{\Delta t} $$
and
$$ \sigma = \frac{S}{\sqrt{\Delta t}}. $$


### European option price

# References

* [Why log returns](https://quantivity.wordpress.com/2011/02/21/why-log-returns/)
* [Drift and volatility of market prices](http://www.lifelong-learners.com/opt/SYL/s1node6.php)
* [Texas A&M maths answer sheet](http://www.math.tamu.edu/~stecher/425/Sp01/exam2.pdf)